### (a)

\begin{align*}
  \langle \Phi_m(x), \Phi_m(y) \rangle
  &= \sum_{m \in M} \Phi_m(x), \Phi_m(y)\\
  &= \sum_{m \in M} \sqrt{\frac{d!}{\prod_i m_i!}} \prod_i x_i^{m_i} \cdot \sqrt{\frac{d!}{\prod_i m_i!}} \prod_i y_i^{m_i}\\
  &= \sum_{m \in M} \sqrt{\frac{d!}{\prod_i m_i!}} (\prod_i x_i^{m_i}  \prod_i y_i^{m_i})\\
  &= \sum_{m \in M} \sqrt{\frac{d!}{\prod_i m_i!}} \prod_i x_i^{m_i}y_i^{m_i}\\
  &= \sum_{m \in M} \sqrt{\frac{d!}{\prod_i m_i!}} \prod_i (x_i \cdot y_i)^{m_i}\\
  &= (x_1 \cdot y_1 + \dots + x_n \cdot y_n)^d\\
  &= \langle x, y\rangle^d
\end{align*}

Basic sketch:

The multinomial sum consists of m+d-1 over d-1 different terms, different basis functions.

## Exercise 12

\begin{align*}
  R(f') 
  &= R(f) R(f') - R(f)\\
  &\leq R(f) \vert R(f') - R(f) \vert \\
  &\leq R(f) \cdot 2 \sup_{f \in F} \vert \hat{R}_m(f) - R(f) \vert \\
\end{align*}


\begin{align*}
  P(\vert \hat{R}_m(f) - R(f) \vert \geq \epsilon)
  &= P(\vert \frac{1}{m} \sum_i \hat{R}_i(f) - R(f) \vert\geq \epsilon )\\
  &\leq 2 \exp{-\frac{2m \epsilon^2}{(b-a)^2}}\\
\end{align*}

Now solve for delta

\begin{align*}
  \delta &= 4n \exp{-\frac{2m \epsilon^2}{(b-a)^2}}\\
  \frac{\delta}{4n} &= \exp{-\frac{2m \epsilon^2}{(b-a)^2}}\\
  \log{\frac{\delta}{4n}} &= -\frac{2m \epsilon^2}{(b-a)^2}\\
  \log{\frac{4n}{\delta}} &= \frac{2m \epsilon^2}{(b-a)^2}\\
  \frac{1}{2m}\log{\frac{4n}{\delta}} &= \frac{\epsilon^2}{(b-a)^2}\\
  \sqrt{\frac{1}{2m}\log{\frac{4n}{\delta}}} &= \frac{\epsilon}{(b-a)}\\
\end{align*}


In [13]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import cho_factor, cho_solve
from scipy.spatial.distance import cdist

In [3]:
data = np.load("diabetes_data.npy", allow_pickle=True).item()
X_train = data["Xtrain"]
Y_train = data["Ytrain"]
X_test = data["Xtest"]
Y_test = data["Ytest"]

In [43]:
def gaussian_kernel(x, y, mu):
    return np.exp(- mu * cdist(x,y)**2) 

In [7]:
def Loss(Y, Y_prediction):
    return 0.5* norm(Y == np.sign(Y_prediction))

In [191]:
def train(X, Y, lambda_reg, mu):
    n = X.shape[0]
    K = gaussian_kernel(X, X, mu)
    alpha = cho_solve(cho_factor(K + n * lambda_reg * np.eye(n)), Y)
    train_loss = Loss(Y, predict(X, X, alpha, mu))
    return alpha, train_loss

def predict(X_pred, X_t, alpha, mu):
    return np.sum(np.multiply(np.squeeze(alpha), gaussian_kernel(X_pred, X_t, mu)), axis=1)
    
def test(X_test, Y_test, X, alpha, mu):
    return Loss(Y_test, predict(X_test, X, alpha, mu))

In [203]:
def CrossValidation(X, Y, lambdas, mus, k=5):
    X_split = np.array(np.array_split(X, k))
    Y_split = np.array(np.array_split(Y, k))
    min_loss=np.inf
    min_mu = None 
    min_lambda = None
    for mu in mus:
        for l in lambdas:
            # compute k-fold cross validation
            validation_loss = []
            for i in range(k):
                x_train = np.concatenate(np.delete(X_split, i, axis=0))
                y_train = np.concatenate(np.delete(Y_split, i, axis=0))
                x_val = X_split[i]
                y_val = Y_split[i]
                alphas, train_loss = train(x_train, y_train, l, mu)
                validation_loss.append(test(x_val, y_val, x_train, alphas, mu)/x_val.shape[0]) # normalized by size of xval
            validation_loss = np.mean(validation_loss)
            print(f"The average loss for mu={mu}, lambda={l} is {validation_loss}")
            # check if smaller
            if validation_loss < min_loss:
                min_loss = validation_loss
                min_mu = mu
                min_lambda = l
                print(f"Found new best parameters mu={min_mu}, lambda={min_lambda}")
    # finally train on the whole set again
    return train(X, Y, min_lambda, min_mu)


In [204]:
mus = [1e-4, 1e-3, 1e-2, 1e-1, 1]
lambdas = [1e-4, 1e-3, 1e-2, 1e-1, 1]
best_alphas, min_test_loss = CrossValidation(X_train, Y_train, lambdas, mus, 5)

The average loss for mu=0.0001, lambda=0.0001 is 0.39796057467662804
Found new best parameters mu=0.0001, lambda=0.0001
The average loss for mu=0.0001, lambda=0.001 is 0.40885305934927574
The average loss for mu=0.0001, lambda=0.01 is 0.40956260094978514
The average loss for mu=0.0001, lambda=0.1 is 0.40956260094978514
The average loss for mu=0.0001, lambda=1 is 0.40956260094978514
The average loss for mu=0.001, lambda=0.0001 is 0.39506631148311
Found new best parameters mu=0.001, lambda=0.0001
The average loss for mu=0.001, lambda=0.001 is 0.39796057467662804
The average loss for mu=0.001, lambda=0.01 is 0.40885305934927574
The average loss for mu=0.001, lambda=0.1 is 0.40956260094978514
The average loss for mu=0.001, lambda=1 is 0.40956260094978514
The average loss for mu=0.01, lambda=0.0001 is 0.39391449353952573
Found new best parameters mu=0.01, lambda=0.0001
The average loss for mu=0.01, lambda=0.001 is 0.3947496429036761
The average loss for mu=0.01, lambda=0.01 is 0.39836541232

In [209]:
min_test_loss

95.33362470817943